<a href="https://colab.research.google.com/github/yutaro-shimizu/GA/blob/main/DeZero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import weakref
import contextlib

class Config:
  enable_backprop = True

@contextlib.contextmanager
def using_config(name, value):
  old_value = getattr(Config, name)
  setattr(Config, name, value)
  try:
    yield
  finally:
    setattr(Config, name, old_value)

def no_grad():
  return using_config('enable_backprop', False)

class Variable:
  __aray_priority__ = 200
  
  def __init__(self, data):
    if data is not None:
      if not isinstance(data,np.ndarray):
        raise TypeError('{} is not supported'.format(type(data)))

    self.data = data
    self.grad = None
    self.creator = None
    self.generation = 0

  def __len__(self):
    return len(self.data)

  def __repr__(self):
    if self.data is None:
      return "variable(None)"
    p = str(self.data).replace("\n", "\n" + " " * 9)
    return "variable(" + p + ")"

  def __mul__(self, other):
    return mul(self, other)

  def set_creator(self, func):
    self.creator = func
    self.generation = func.generation + 1

  def backward(self, retain_grad=False):
    if self.grad is None:
      self.grad = np.ones_like(self.data)

    funcs = []
    seen_set = set()

    def add_func(f):
      """append functions by generations in the calculuation graph"""
      if f not in seen_set:
        funcs.append(f)
        seen_set.add(f)
        funcs.sort(key=lambda x:x.generation)

    add_func(self.creator)
      
    funcs = [self.creator]
    while funcs:
      f = funcs.pop()
      gys = [output().grad for output in f.outputs]
      gxs = f.backward(*gys)
      if not isinstance(gxs,tuple):
        gxs = (gxs,)

      for x, gx in zip(f.inputs, gxs):
        if x.grad is None:
          x.grad = gx
        else:
          x.grad = x.grad + gx

        if x.creator is not None:
          add_func(x.creator)

      if not retain_grad:
        for y in f.outputs:
          y().grad = None
  
  def cleargrad(self):
    self.grad = None

  @property
  def shape(self):
    return self.data.shape
  
  @property
  def ndim(self):
    return self.data.ndim
  
  @property
  def size(self):
    return self.data.size

  @property
  def dtype(self):
    return self.data.dtype

"""helper function to transform input to ndarray instance"""
def as_array(x):
  if np.isscalar(x):
    return np.array(x)
  return (x)

"""helper function to return a variable object"""
def as_variable(obj):
  if isinstance(obj, Variable):
    return obj
  return Variable(obj)

class Function:
  def __call__(self,*inputs):
    # x = input.data
    # y = self.forward(x) #x**2
    # output = Variable(as_array(y))
    # output.set_creator(self)
    # self.input = input
    # self.output = output
    # return output 
    inputs = [as_variable(x) for x in inputs]
    xs = [x.data for x in inputs]
    ys = self.forward(*xs)
    if not isinstance(ys,tuple):
      ys = (ys,)
    outputs = [Variable(as_array(y)) for y in ys]

    if Config.enable_backprop:
      self.generation = max([x.generation for x in inputs])
      for output in outputs:
        output.set_creator(self)
      self.inputs = inputs
      self.outputs = [weakref.ref(output) for output in outputs]
    return outputs if len(outputs) > 1 else outputs[0]

  def forward(self,x):
    raise NotImplementedError()

  def backward(self, gy):
    raise NotImplementedError()

class Square(Function):
  def forward(self,x):
    return x**2

  def backward(self, gy):
    x = self.inputs[0].data
    gx = 2 *x *gy
    return gx

class Exp(Function):
  def forward(self, x):
    return np.exp(x)

  def backward(self,gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

class Add(Function):
  def forward(self,x0, x1):
    return x0 + x1

  def backward(self, gy):
    return gy, gy

class Mul(Function):
  def forward(self, x0, x1):
    y = x0 * x1
    return y

  def backward(self, gy):
    x0, x1 = self.inputs[0].data, self.inputs[1].data
    return gy * x1, gy *x0

def square(x):
  # f = Square()
  return Square()(x)

def exp(x):
  # f = Exp()
  return Exp()(x)

def add(x0, x1):
  return Add()(x0,x1)

def mul(x0, x1):
  return Mul()(x0,x1)

def setup_variable():
  Variable.__add__ = add
  Variable.__radd = add
  Variable.__mul__ = mul
  Variable.__rmul__ = mul

In [ ]:
x0 = Variable(np.array(2))
x1 = Variable(np.array(3))
f = Add()
y = f(x0, x1)
print(y.data)

with using_config('enable_backprop', False):
  x = Variable(np.array(2.0))
  y = square(x)


x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x)

5
variable([[1 2 3]
          [4 5 6]])


In [ ]:
import unittest

def numerical_diff(f,x,eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data) / 2*eps

class SquareTest(unittest.TestCase):
  def test_forward(self):
    x = Variable(np.array(2.0))
    y = square(x)
    expected = np.array(4.0)
    self.assertEqual(y.data,expected)

  def test_gradient_check(self):
    x = Variable(np.random.rand(1))
    y = square(x)
    y.backward()
    num_grad = numerical_diff(square,x)
    flg = np.allclose(x.grad,num_grad)
    self.assertTrue(flg)

unittest.main()